# Types of Agent.

# Agents 🤖
Agents are like "tools" for LLMs. They allow a LLM to access Google search, perform complex calculations with Python, and even make SQL queries.


## What is an agent?
Definition: The key behind agents is giving LLM's the possibility of using tools in their workflow. This is where langchain departs from the popular chatgpt implementation and we can start to get a glimpse of what it offers us as builders. Until now, we covered several building blocks in isolation. Let's see them come to life.

The official definition of agents is the following:

  * Agents use an LLM to determine which actions to take and in what order. An action can either be using a tool and observing its output, or returning to the user.

In this edition we will cover what we may call 'generic' agents which really able to perform many meta tasks. There are other more specific agents that are tuned for different tasks (called 'agent-toolkits').


## 4 Types of agent:
1. zero-shot-react-description,
2. conversational-react-description,
3. react-docstore,
4. self-ask-with-search.


Valid Types:
1. AgentType.ZERO_SHOT_REACT_DESCRIPTION: 'zero-shot-react-description',
2. AgentType.REACT_DOCSTORE: 'react-docstore'
3. AgentType.SELF_ASK_WITH_SEARCH: 'self-ask-with-search'
4. AgentType.CONVERSATIONAL_REACT_DESCRIPTION: 'conversational-react-description'
5. AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION: 'chat-zero-shot-react-description'
6. AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION: 'chat-conversational-react-description'
7. AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION: 'structured-chat-zero-shot-react-description'
8. AgentType.OPENAI_FUNCTIONS: 'openai-functions'
9. AgentType.OPENAI_MULTI_FUNCTIONS: 'openai-multi-functions'

---

In [ ]:
!pip install --quiet langchain langchain_openai langchain_community google-search-results wikipedia sqlalchemy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 26.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.6/315.6 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.4 MB/s eta 0:00:00


In [ ]:
import os
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [ ]:
from langchain_openai import OpenAI

llm = OpenAI(
    temperature=0
)

# Count the number of tokens used after each call

In [ ]:
from langchain.callbacks import get_openai_callback

def count_tokens(agent, query):
    with get_openai_callback() as cb:
        result = agent(query)
        print(f'Spent a total of {cb.total_tokens} tokens')

    return result

# Create a dummy database to create SQL tool

In [ ]:
from sqlalchemy import MetaData

metadata_obj = MetaData()


from sqlalchemy import Column, Integer, String, Table, Date, Float

stocks = Table(
    "stocks",
    metadata_obj,
    Column("obs_id", Integer, primary_key=True),
    Column("stock_ticker", String(4), nullable=False),
    Column("price", Float, nullable=False),
    Column("date", Date, nullable=False),
)



from sqlalchemy import create_engine

engine = create_engine("sqlite:///:memory:")
metadata_obj.create_all(engine)



from datetime import datetime

observations = [
    [1, 'ABC', 200, datetime(2023, 1, 1)],
    [2, 'ABC', 208, datetime(2023, 1, 2)],
    [3, 'ABC', 232, datetime(2023, 1, 3)],
    [4, 'ABC', 225, datetime(2023, 1, 4)],
    [5, 'ABC', 226, datetime(2023, 1, 5)],
    [6, 'XYZ', 810, datetime(2023, 1, 1)],
    [7, 'XYZ', 803, datetime(2023, 1, 2)],
    [8, 'XYZ', 798, datetime(2023, 1, 3)],
    [9, 'XYZ', 795, datetime(2023, 1, 4)],
    [10, 'XYZ', 791, datetime(2023, 1, 5)],
]




from sqlalchemy import insert

def insert_obs(obs):
    stmt = insert(stocks).values(
    obs_id=obs[0],
    stock_ticker=obs[1],
    price=obs[2],
    date=obs[3]
    )

    with engine.begin() as conn:
        conn.execute(stmt)




for obs in observations:
    insert_obs(obs)

# Agent types

Using one of langchain's pre-built agents involves three variables:

1. defining the tools or the toolkit
2. defining the llm
3. defining the agent type

This is all really easy to do in langchain.


# Agent type

##1: Zero Shot React
In this first example we will use slightly different type of agent - SQL Agent which can be instantiated with it's own method create_sql_agent. Other agents will be instantiated in more generic way as we will see below in other examples.

This method uses toolkit instead of simple list of tools. You can read more about them in the documentation.

For this use case, we will use SQLDatabaseToolkit.

As the name suggests, we will use this agent to perform 'zero shot' tasks on the input. That means that we will not have several, interdependent interactions but only one. In other words, this agent will have no memory.

Now we are ready to initialize the agent! We will use verbose in True so we can see what is our agent's 'thinking' process.

Important Note: When interacting with agents it is really important to set the max_iterations parameters because agents can get stuck in infinite loops that consume plenty of tokens. The default value is 15 to allow for many tools and complex reasoning but for most applications you should keep it much lower.

# Create SQLChain


We are installing the `langchain_experimental` library here, since the SQLDatabaseChain is located there. This might be changed in the future and moved into official langchain library.

[SQLChain](https://api.python.langchain.com/en/latest/sql/langchain_experimental.sql.base.SQLDatabaseChain.html)

In [ ]:
!pip install langchain_experimental -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.5/202.5 kB 1.4 MB/s eta 0:00:00


In [ ]:
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

In [ ]:
db = SQLDatabase(engine)

# Create SQL Chain
sql_chain = SQLDatabaseChain(
    llm = llm,
    database = db,
    verbose=True
)

/usr/local/lib/python3.10/dist-packages/langchain_experimental/sql/base.py:77: UserWarning: Directly instantiating an SQLDatabaseChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


# Create SQL Agent

In [ ]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType

agent_executor = create_sql_agent(
    llm = llm,
    toolkit=SQLDatabaseToolkit(db = db,
                               llm = llm),
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    max_iterations=3

)

In [ ]:
result = count_tokens(
    agent_executor,
    "What is the multiplication of the ratio between stock " +
    "prices for 'ABC' and 'XYZ' in January 3rd and the ratio " +
    "between the same stock prices in January the 4th?"
)



> Entering new SQL Agent Executor chain...
 We need to get the stock prices for 'ABC' and 'XYZ' on January 3rd and January 4th, and then calculate the ratio between them.
Action: sql_db_query
Action Input: SELECT price FROM stocks WHERE stock_name = 'ABC' AND date = 'January 3rd'Error: (sqlite3.OperationalError) no such column: stock_name
[SQL: SELECT price FROM stocks WHERE stock_name = 'ABC' AND date = 'January 3rd']
(Background on this error at: https://sqlalche.me/e/20/e3q8)We need to use sql_db_schema to get the correct table fields.
Action: sql_db_schema
Action Input: stocks
CREATE TABLE stocks (
	obs_id INTEGER NOT NULL, 
	stock_ticker VARCHAR(4) NOT NULL, 
	price FLOAT NOT NULL, 
	date DATE NOT NULL, 
	PRIMARY KEY (obs_id)
)

/*
3 rows from stocks table:
obs_id	stock_ticker	price	date
1	ABC	200.0	2023-01-01
2	ABC	208.0	2023-01-02
3	ABC	232.0	2023-01-03
*/Now we can use the correct table field 'stock_ticker' in our query.
Action: sql_db_query
Action Input: SELECT price FROM st

# Check the prompt

In [ ]:
print(agent_executor.agent)

runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_str(x['intermediate_steps']))
})
| PromptTemplate(input_variables=['agent_scratchpad', 'input'], partial_variables={'tools': "sql_db_query - Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.\nsql_db_schema - Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3\nsql_db_list_tables - Input is an empty string, output is a comma-separated list of tables in the database.\nsql_db_query_checker - Use this tool to double check if you

In [ ]:
print(agent_executor.agent.runnable)

first=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_str(x['intermediate_steps']))
}) middle=[PromptTemplate(input_variables=['agent_scratchpad', 'input'], partial_variables={'tools': "sql_db_query - Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.\nsql_db_schema - Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3\nsql_db_list_tables - Input is an empty string, output is a comma-separated list of tables in the database.\nsql_db_query_checker - Use this tool to double check if 

In [ ]:
from pprint import pprint
pprint(agent_executor.agent.runnable.middle)

[PromptTemplate(input_variables=['agent_scratchpad', 'input'], partial_variables={'tools': "sql_db_query - Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.\nsql_db_schema - Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3\nsql_db_list_tables - Input is an empty string, output is a comma-separated list of tables in the database.\nsql_db_query_checker - Use this tool to double check if your query is correct before executing it. Always use this tool before executing a query with sql_db_query!", 'tool_names': 'sql_

In [ ]:
from pprint import pprint
pprint(agent_executor.agent.runnable.middle[0].partial_variables)

{'tool_names': 'sql_db_query, sql_db_schema, sql_db_list_tables, '
               'sql_db_query_checker',
 'tools': 'sql_db_query - Input to this tool is a detailed and correct SQL '
          'query, output is a result from the database. If the query is not '
          'correct, an error message will be returned. If an error is '
          'returned, rewrite the query, check the query, and try again. If you '
          "encounter an issue with Unknown column 'xxxx' in 'field list', use "
          'sql_db_schema to query the correct table fields.\n'
          'sql_db_schema - Input to this tool is a comma-separated list of '
          'tables, output is the schema and sample rows for those tables. Be '
          'sure that the tables actually exist by calling sql_db_list_tables '
          'first! Example Input: table1, table2, table3\n'
          'sql_db_list_tables - Input is an empty string, output is a '
          'comma-separated list of tables in the database.\n'
          'sql_

In [ ]:
import langchain
print(langchain.__version__)

0.2.5


In [ ]:
from langchain.agents import load_tools

# prebuilt tool
tools = load_tools(
    ['llm-math'],
    llm = llm
)

# append custom tool
tools.append(sql_chain)


# initialize agent
from langchain.agents import initialize_agent

zero_shot_agent = initialize_agent(
    # agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION, # or
    agent = "zero-shot-react-description",
    tools = tools,
    llm = llm,
    verbose = True,
    max_iteration = 3
)

In [ ]:
result = zero_shot_agent(
    "What is the multiplication of the ratio between stock prices for 'ABC' "
    "and 'XYZ' in January 3rd and the ratio between the same stock prices in "
    "January the 4th?"
)

#Agent type
##2: Conversational React

The zero shot agent is really interesting but, as we said before, it has no memory. What if we want an assistant that remembers things we have talked about and can also reason about them and use tools? For that we have the conversational react agent.

We will use the math tool in this example and load it as below:

In [ ]:
from langchain.agents import load_tools

# initialize chain
tools = load_tools(
    ['llm-math'],
    llm = llm
)


In [ ]:
# Adding Memory
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key='chat_history')

In [ ]:
from re import VERBOSE
from langchain.agents import initialize_agent

# initialize agent
conversation_agent = initialize_agent(
    agent = AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    tools=tools,
    llm = llm,
    verbose = True,
    max_iterations = 3,
    memory=memory
)

In [ ]:
# run agent
result = count_tokens(
    conversation_agent,
    "What's the result of an investment of $10,000 growing at 8% annually for 5 years with compound interest?"
)



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? Yes
Action: Calculator
Action Input: $10,000, 8%, 5 years, compound interest
Observation: Answer: 14693.280768000006
Thought: Do I need to use a tool? No
AI: The result of an investment of $10,000 growing at 8% annually for 5 years with compound interest is $14,693.28.

> Finished chain.
Spent a total of 1152 tokens


In [ ]:
# Check prompt
print(conversation_agent.agent.llm_chain.prompt.template)

Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful tool that can help with a wide range of tasks 

#Agent type

##3: React Docstore
This type of agent is similar to the ones we have seen so far but it includes the interaction with a docstore.

It will have two and only two tools at its disposal:
1. 'Search' and,
2. 'Lookup'.

With 'Search' it will bring up a relevant article and with 'Lookup' the agent will find the right piece of information in the article.

In [ ]:
from langchain import Wikipedia
from langchain.agents.react.base import DocstoreExplorer, Tool

In [ ]:
docstore = DocstoreExplorer(Wikipedia())

# Initialize tool
tools = [
    Tool(
        name='Search',
        description='Search Wikipedia',
        func=docstore.search
    ),
    Tool(
        name = "Lookup",
        description='Look up for a sentence in Wikipedia',
        func=docstore.lookup
    )
]

In [ ]:
# Initialize Agent
docstore_agent = initialize_agent(
    tools,
    llm,
    agent='react-docstore',
    verbose='True',
    max_iteration=3
)

In [ ]:
count_tokens(docstore_agent, "What was the last book Abdul Kalam write and when?")



> Entering new AgentExecutor chain...
Thought: I need to search Abdul Kalam, find his books, then find the last book he wrote and when it was written.
Action: Search[Abdul Kalam]
Observation: Could not find [Abdul Kalam]. Similar: ['A. P. J. Abdul Kalam', 'Abdul Kalam Island', 'APJ Abdul Kalam Technological University', 'Dr. A. P. J. Abdul Kalam Award', 'Maulana Azad', 'Dr. A.P.J. Abdul Kalam Memorial', 'List of awards and honours received by A. P. J. Abdul Kalam', 'Syed Abdul Nazeer', 'I Am Kalam', 'Abdul Kalam (Varanasi politician)']
Thought: To find the books written by Abdul Kalam, I can search A. P. J. Abdul Kalam.
Action: Search[A. P. J. Abdul Kalam]
Observation: Avul Pakir Jainulabdeen Abdul Kalam BR ( ; 15 October 1931 – 27 July 2015) was an Indian aerospace scientist and statesman who served as the 11th president of India from 2002 to 2007. Born and raised in a Muslim family in Rameswaram, Tamil Nadu, he studied physics and aerospace engineering. He spent the next four decad

{'input': 'What was the last book Abdul Kalam write and when?',
 'output': 'Transcendence: My Spiritual Experiences with Pramukh Swamiji'}

#Agent type
##4: Self Ask with Search
This is the first-choice agent to use when using LLM's to extract information with a search engine.

The agent will ask follow-up questions and use the search functionality to get intermediate answers that help it get to a final answer.

In [ ]:
from google.colab import userdata
os.environ["SERPAPI_API_KEY"] = userdata.get('SERPAPI')

In [ ]:
from langchain.agents import AgentType, Tool, initialize_agent
from langchain_community.utilities import SerpAPIWrapper
from langchain_openai import OpenAI

llm = OpenAI(temperature=0)

# Create Chain/Wrapper
params = {
    "engine": "google",
    "gl": "us",
    "hl": "en",
}

search = SerpAPIWrapper(params=params)


# Create Tool
tools = [
    Tool(
        name="Intermediate Answer",
        func=search.run,
        description="useful for when you need to ask with search",
    )
]

# Initialize Agent
self_ask_with_search = initialize_agent(
    tools,
    llm,
    agent=AgentType.SELF_ASK_WITH_SEARCH,
    verbose=True,
    max_iteration=3,
    handle_parsing_errors = True
)

# Run Agent
self_ask_with_search.run("Who won India Pakistan Match?")



> Entering new AgentExecutor chain...
 Yes.
Follow up: When was the India Pakistan match played?
Intermediate answer: {'title': 'India vs Pakistan', 'games': [{'tournament': "ICC Men's T20 World Cup", 'stadium': 'Nassau County International Cricket Stadium', 'stadium_kgmid': '/g/11vpfvhl5s', 'stage': 'Group stage · T20 19 of 55', 'date': 'Sun, Jun 9', 'teams': [{'name': 'India', 'score': '119 (19)', 'kgmid': '/m/024nj1', 'thumbnail': 'https://ssl.gstatic.com/onebox/media/sports/logos/mlXOOB9HXxLpoeS2dHSgGA_48x48.png'}, {'name': 'Pakistan', 'score': '113/7 (20)', 'kgmid': '/m/038_0z', 'thumbnail': 'https://ssl.gstatic.com/onebox/media/sports/logos/-I9tpg9uetJjQNCh15jmKw_48x48.png'}], 'status': 'IND won by 6 runs'}, {'tournament': 'ICC Cricket World Cup', 'stadium': 'Narendra Modi Stadium', 'stadium_kgmid': '/m/06m73y', 'stage': 'ODI 12 of 48', 'date': 'Oct 14, 23', 'teams': [{'name': 'Pakistan', 'score': '191 (42.5)', 'kgmid': '/m/038_0z', 'thumbnail': 'https://ssl.gstatic.com/onebox/

'India'